# Quality Evaluators with the Azure AI Evaluation SDK
The following sample shows the basic way to evaluate a Generative AI application in your development environment with the Azure AI evaluation SDK.

> ✨ ***Note*** <br>
> Please check the reference document before you get started - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk

## 🔨 Current Support and Limitations (as of 2025-01-14) 
- Check the region support for the Azure AI Evaluation SDK. https://learn.microsoft.com/en-us/azure/ai-studio/concepts/evaluation-metrics-built-in?tabs=warning#region-support

### Region support for evaluations
| Region              | Hate and Unfairness, Sexual, Violent, Self-Harm, XPIA, ECI (Text) | Groundedness (Text) | Protected Material (Text) | Hate and Unfairness, Sexual, Violent, Self-Harm, Protected Material (Image) |
|---------------------|------------------------------------------------------------------|---------------------|----------------------------|----------------------------------------------------------------------------|
| North Central US    | no                                                               | no                  | no                         | yes                                                                        |
| East US 2           | yes                                                              | yes                 | yes                        | yes                                                                        |
| Sweden Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| US North Central    | yes                                                              | no                  | yes                        | yes                                                                        |
| France Central      | yes                                                              | yes                 | yes                        | yes                                                                        |
| Switzerland West    | yes                                                              | no                  | no                         | yes                                                                        |

### Region support for adversarial simulation
| Region            | Adversarial Simulation (Text) | Adversarial Simulation (Image) |
|-------------------|-------------------------------|---------------------------------|
| UK South          | yes                           | no                              |
| East US 2         | yes                           | yes                             |
| Sweden Central    | yes                           | yes                             |
| US North Central  | yes                           | yes                             |
| France Central    | yes                           | no                              |


## ✔️ Pricing and billing
- Effective 1/14/2025, Azure AI Safety Evaluations will no longer be free in public preview. It will be billed based on consumption as following:

| Service Name              | Safety Evaluations       | Price Per 1K Tokens (USD) |
|---------------------------|--------------------------|---------------------------|
| Azure Machine Learning    | Input pricing for 3P     | $0.02                     |
| Azure Machine Learning    | Output pricing for 3P    | $0.06                     |
| Azure Machine Learning    | Input pricing for 1P     | $0.012                    |
| Azure Machine Learning    | Output pricing for 1P    | $0.012                    |


In [ ]:
import pandas as pd
import os
import json

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import RelevanceEvaluator
from azure.ai.evaluation import GroundednessEvaluator, GroundednessProEvaluator
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation,
    Dataset,
    EvaluatorConfiguration,
    ConnectionType,
    EvaluationSchedule,
    RecurrenceTrigger,
    ApplicationInsightsConfiguration
)
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
    F1ScoreEvaluator,
    RetrievalEvaluator
)



load_dotenv("../.env")

In [2]:
credential = DefaultAzureCredential()

azure_ai_project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ.get("AZURE_AI_PROJECT_CONN_STR"),  # At the moment, it should be in the format "<Region>.api.azureml.ms;<AzureSubscriptionId>;<ResourceGroup>;<HubName>" Ex: eastus2.api.azureml.ms;xxxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxxxxx;rg-sample;sample-project-eastus2
)

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "type": "azure_openai",
}

In [3]:
query_response = dict(
    query="Which tent is the most waterproof?",
    context="The Alpine Explorer Tent is the most water-proof of all tents available.",
    response="The Alpine Explorer Tent is the most waterproof."
)

conversation_str =  """{"messages": [ { "content": "Which tent is the most waterproof?", "role": "user" }, { "content": "The Alpine Explorer Tent is the most waterproof", "role": "assistant", "context": "From the our product list the alpine explorer tent is the most waterproof. The Adventure Dining Table has higher weight." }, { "content": "How much does it cost?", "role": "user" }, { "content": "$120.", "role": "assistant", "context": "The Alpine Explorer Tent is $120."} ] }""" 
conversation = json.loads(conversation_str)

## 🧪 AI-assisted Groundedness evaluator
- Prompt-based groundedness using your own model deployment to output a score and an explanation for the score is currently supported in all regions.
- Groundedness Pro evaluator leverages Azure AI Content Safety Service (AACS) via integration into the Azure AI Foundry evaluations. No deployment is required, as a back-end service will provide the models for you to output a score and reasoning. Groundedness Pro is currently supported in the East US 2 and Sweden Central regions.

In [ ]:
# Initialzing Groundedness and Groundedness Pro evaluators
groundedness_eval = GroundednessEvaluator(model_config)
# No need to set the model_config for GroundednessProEvaluator

query_response = dict(
    query="Which tent is the most waterproof?", # optional
    context="The Alpine Explorer Tent is the most water-proof of all tents available.",
    response="The Alpine Explorer Tent is the most waterproof."
)

# query_response = dict(
#     query="어떤 텐트가 방수 기능이 있어?", # optional
#     context="알파인 익스플로러 텐트가 모든 텐트 중 가장 방수 기능이 뛰어남",
#     response="알파인 익스플로러 텐트가 방수 기능이 있습니다."
# )

# Running Groundedness Evaluator on a query and response pair
groundedness_score = groundedness_eval(
    **query_response
)
print(groundedness_score)

# input conversation result for the groundedness evaluator
groundedness_conv_score = groundedness_eval(conversation=conversation)
print(groundedness_conv_score)

## 🧪 AI-assisted RelevanceEvaluator
- Relevance refers to how effectively a response addresses a question. It assesses the accuracy, completeness, and direct relevance of the response based solely on the given information.

In [ ]:
relevance_eval = RelevanceEvaluator(model_config)

query_response = dict(
    query="Which tent is the most waterproof?",
    #context="The Alpine Explorer Tent is the most water-proof of all tents available.",
    response="The Alpine Explorer Tent is the most waterproof."
)

relevance_score = relevance_eval(
    **query_response
)
print(relevance_score)

# input conversation result
relevance_conv_score = relevance_eval(conversation=conversation)
print(relevance_conv_score)

## 🧪 AI-assisted CoherenceEvaluator
- Coherence refers to the logical and orderly presentation of ideas in a response, allowing the reader to easily follow and understand the writer's train of thought. A coherent answer directly addresses the question with clear connections between sentences and paragraphs, using appropriate transitions and a logical sequence of ideas.

In [ ]:
coherence_eval = CoherenceEvaluator(model_config)

query_response = dict(
    query="Which tent is the most waterproof?",
    #context="The Alpine Explorer Tent is the most water-proof of all tents available.",
    response="The Alpine Explorer Tent is the most waterproof."
)

relevance_score = relevance_eval(
    **query_response
)
print(relevance_score)

relevance_conv_score = relevance_eval(conversation=conversation)
print(relevance_conv_score)

coherence_score = coherence_eval(
    **query_response
)
print(coherence_score)

# input conversation result
coherence_conv_score = coherence_eval(conversation=conversation)
print(coherence_conv_score)

## 🧪 AI-assisted FluencyEvaluator
- Fluency refers to the effectiveness and clarity of written communication, focusing on grammatical accuracy, vocabulary range, sentence complexity, coherence, and overall readability. It assesses how smoothly ideas are conveyed and how easily the text can be understood by the reader.

In [ ]:
fluency_eval = FluencyEvaluator(model_config)

query_response = dict(
    #query="Which tent is the most waterproof?",
    #context="The Alpine Explorer Tent is the most water-proof of all tents available.",
    response="The Alpine Explorer Tent is the most waterproof."
)

fluency_score = fluency_eval(
    **query_response
)
print(fluency_score)

# input conversation result
fluency_conv_score = fluency_eval(conversation=conversation)
print(fluency_conv_score)

## 🧪 AI-assisted SimilarityEvaluator
- The similarity metric is calculated by instructing a language model to follow the definition (in the description) and a set of grading rubrics, evaluate the user inputs, and output a score on a 5-point scale (higher means better quality). See the definition and grading rubrics below.
- The recommended scenario is NLP tasks with a user query. Use it when you want an objective evaluation of an AI model's performance, particularly in text generation tasks where you have access to ground truth responses. Similarity enables you to assess the generated text's semantic alignment with the desired content, helping to gauge the model's quality and accuracy.

In [ ]:
similarity_eval = SimilarityEvaluator(model_config)


query_response = dict(
    query="Which tent is the most waterproof?",
    #context="The Alpine Explorer Tent is the most water-proof of all tents available.",
    response="The Alpine Explorer Tent is the most waterproof.",
    ground_truth="The Alpine Explorer tent has a rainfly waterproof rating of 2000mm"
)

similarity_score = similarity_eval(
    **query_response
)
print(similarity_score)

# input conversation Not support
# similarity_conv_score = similarity_eval(conversation=conversation)
# print(similarity_conv_score)

## 🚀 Run Evaluators in Azure Cloud

In [61]:
# id for each evaluator can be found in your AI Studio registry - please see documentation for more information
# init_params is the configuration for the model to use to perform the evaluation
# data_mapping is used to map the output columns of your query to the names required by the evaluator
# Evaluator parameter format - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk#evaluator-parameter-format
evaluators_cloud = {
    "f1_score": EvaluatorConfiguration(
        id=F1ScoreEvaluator.id,
    ),
    "relevance": EvaluatorConfiguration(
        id=RelevanceEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "context": "${data.context}", "response": "${data.response}"},
    ),
    "groundedness": EvaluatorConfiguration(
        id=GroundednessEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "context": "${data.context}", "response": "${data.response}"},
    ),
    # "retrieval": EvaluatorConfiguration(
    #     #from azure.ai.evaluation._evaluators._common.math import list_mean_nan_safe\nModuleNotFoundError: No module named 'azure.ai.evaluation._evaluators._common.math'
    #     #id=RetrievalEvaluator.id,
    #     id="azureml://registries/azureml/models/Retrieval-Evaluator/versions/2",
    #     init_params={"model_config": model_config},
    #     data_mapping={"query": "${data.query}", "context": "${data.context}", "response": "${data.response}"},
    # ),
    "coherence": EvaluatorConfiguration(
        id=CoherenceEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "response": "${data.response}"},
    ),
    "fluency": EvaluatorConfiguration(
        id=FluencyEvaluator.id,
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "context": "${data.context}", "response": "${data.response}"},
    ),
     "similarity": EvaluatorConfiguration(
        # currently bug in the SDK, please use the id below
        #id=SimilarityEvaluator.id,
        id="azureml://registries/azureml/models/Similarity-Evaluator/versions/3",
        init_params={"model_config": model_config},
        data_mapping={"query": "${data.query}", "response": "${data.response}"},
    ),

}


### Data
- The following code demonstrates how to upload the data for evaluation to your Azure AI project. Below we use evaluate_test_data.jsonl which exemplifies LLM-generated data in the query-response format expected by the Azure AI Evaluation SDK. For your use case, you should upload data in the same format, which can be generated using the Simulator from Azure AI Evaluation SDK.

- Alternatively, if you already have an existing dataset for evaluation, you can use that by finding the link to your dataset in your registry or find the dataset ID.

In [ ]:
# # Upload data for evaluation
data_id, _ = azure_ai_project_client.upload_file("../data/evaluate_test_data.jsonl")
# data_id = "azureml://registries/<registry>/data/<dataset>/versions/<version>"
# To use an existing dataset, replace the above line with the following line
# data_id = "<dataset_id>"

### Configure Evaluators to Run
- The code below demonstrates how to configure the evaluators you want to run. In this example, we use the F1ScoreEvaluator, RelevanceEvaluator and the ViolenceEvaluator, but all evaluators supported by Azure AI Evaluation are supported by cloud evaluation and can be configured here. You can either import the classes from the SDK and reference them with the .id property, or you can find the fully formed id of the evaluator in the AI Studio registry of evaluators, and use it here. 

In [64]:
evaluation = Evaluation(
    display_name="Cloud Evaluation",
    description="Cloud Evaluation of dataset",
    data=Dataset(id=data_id),
    evaluators=evaluators_cloud,
)

# Create evaluation
evaluation_response = azure_ai_project_client.evaluations.create(
    evaluation=evaluation,
)

In [ ]:
# Get evaluation
get_evaluation_response = azure_ai_project_client.evaluations.get(evaluation_response.id)

print("----------------------------------------------------------------")
print("Created evaluation, evaluation ID: ", get_evaluation_response.id)
print("Evaluation status: ", get_evaluation_response.status)
print("AI Foundry Portal URI: ", get_evaluation_response.properties["AiStudioEvaluationUri"])
print("----------------------------------------------------------------")

In [66]:
from tqdm import tqdm
import time

# Monitor the status of the run_result
def monitor_status(project_client:AIProjectClient, evaluation_response_id:str):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = project_client.evaluations.get(evaluation_response_id).status
        if status == "Queued":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = project_client.evaluations.get(evaluation_response_id).status
        while(pbar.n < 3):
            pbar.update(1)
        print("Operation Completed")

In [ ]:
monitor_status(azure_ai_project_client, evaluation_response.id)

## 🧪 Evaluate in the Cloud on a Schedule with Online Evaluation
- You can configure the RecurrenceTrigger based on the class definition here. The code below demonstrates how to configure the RecurrenceTrigger to run the evaluation every 24 hours. You can also configure the trigger to run at a different interval, or at a specific time of day. Check the Trace menu in the Azure AI Foundry to see the results of the evaluation.
- reference: https://learn.microsoft.com/en-us/azure/ai-studio/how-to/online-evaluation

In [17]:
from datetime import datetime

# Your Application Insights resource ID
# At the moment, it should be something in the format "/subscriptions/<AzureSubscriptionId>/resourceGroups/<ResourceGroup>/providers/Microsoft.Insights/components/<ApplicationInsights>""
app_insights_resource_id = os.environ.get("APP_INSIGHTS_RESOURCE_ID")

today = datetime.now()

# Name of your generative AI application (will be available in trace data in Application Insights)
service_name = os.environ.get("SERVICE_NAME") + "_" + today.strftime("%Y-%m-%d-%H-%M-%S")

evaluation_name = os.environ.get("EVALUATION_NAME") + "_" + today.strftime("%Y-%m-%d-%H-%M-%S")


In [18]:
kusto_query = 'let gen_ai_spans=(dependencies | where isnotnull(customDimensions["gen_ai.system"]) | extend response_id = tostring(customDimensions["gen_ai.response.id"]) | project id, operation_Id, operation_ParentId, timestamp, response_id); let gen_ai_events=(traces | where message in ("gen_ai.choice", "gen_ai.user.message", "gen_ai.system.message") or tostring(customDimensions["event.name"]) in ("gen_ai.choice", "gen_ai.user.message", "gen_ai.system.message") | project id= operation_ParentId, operation_Id, operation_ParentId, user_input = iff(message == "gen_ai.user.message" or tostring(customDimensions["event.name"]) == "gen_ai.user.message", parse_json(iff(message == "gen_ai.user.message", tostring(customDimensions["gen_ai.event.content"]), message)).content, ""), system = iff(message == "gen_ai.system.message" or tostring(customDimensions["event.name"]) == "gen_ai.system.message", parse_json(iff(message == "gen_ai.system.message", tostring(customDimensions["gen_ai.event.content"]), message)).content, ""), llm_response = iff(message == "gen_ai.choice", parse_json(tostring(parse_json(tostring(customDimensions["gen_ai.event.content"])).message)).content, iff(tostring(customDimensions["event.name"]) == "gen_ai.choice", parse_json(parse_json(message).message).content, "")) | summarize operation_ParentId = any(operation_ParentId), Input = maxif(user_input, user_input != ""), System = maxif(system, system != ""), Output = maxif(llm_response, llm_response != "") by operation_Id, id); gen_ai_spans | join kind=inner (gen_ai_events) on id, operation_Id | project Input, System, Output, operation_Id, operation_ParentId, gen_ai_response_id = response_id'

# AzureMSIClientId is the clientID of the User-assigned managed identity created during set-up - see documentation for how to find it
properties = {"AzureMSIClientId": os.environ.get("AZURE_MSI_CLIENT_ID")}

# Connect to your Application Insights resource
app_insights_config = ApplicationInsightsConfiguration(
    resource_id=app_insights_resource_id, query=kusto_query, service_name=service_name
)

In [ ]:
# Frequency to run the schedule
recurrence_trigger = RecurrenceTrigger(frequency="day", interval=1)

# Configure the online evaluation schedule
evaluation_schedule = EvaluationSchedule(
    data=app_insights_config,
    evaluators=evaluators_cloud,
    trigger=recurrence_trigger,
    description=f"{service_name} evaluation schedule",
    properties=properties
)

# Create the online evaluation schedule
created_evaluation_schedule = azure_ai_project_client.evaluations.create_or_replace_schedule(service_name, evaluation_schedule)
print(
    f"Successfully submitted the online evaluation schedule creation request - {created_evaluation_schedule.name}, currently in {created_evaluation_schedule.provisioning_state} state."
)

evaluation_schedule = azure_ai_project_client.evaluations.get_schedule(service_name)
print(evaluation_schedule)

# Sample for list evaluation schedules
for evaluation_schedule in azure_ai_project_client.evaluations.list_schedule():
    print(evaluation_schedule)

# Sample for disable an evaluation schedule with name
# project_client.evaluations.disable_schedule(service_name)